In [1]:
import pandas as pd
import json
import os
import csv
from datasets import load_dataset
import numpy as np

## GPA2Bench Dataset

In [25]:
#GPA2Bench Dataset
data = []
directory = 'GPA2Bench/GPA2Bench_PHX' #change to whichever directory
for file in os.listdir(directory):
    file_df = pd.read_json(os.path.join(directory, file)).transpose()

    if file == 'ground.json':
        file_df['label'] = 'authentic'
        file_df['prompt'] = 'not applicable'
    else:
        file_name = os.path.splitext(file)[0]
        prompt = file_name.split('_')[-1]

        file_df['label'] = 'synthetic'
        file_df['prompt'] = prompt

    data.append(file_df)

df = pd.concat(data, ignore_index=True)
df.to_csv(f'{directory}/GPA2Bench_PHX.csv', index=False)


          id                                              title  \
0  0704.0001  calculation of prompt diphoton production cros...   
1  0704.0135  a single trapped ion as a time-dependent harmo...   
2  0704.0212  curvature and isocurvature perturbations in tw...   
3  0704.0240   viscosity, black holes, and quantum field theory   
4  0704.0245             one-loop mhv rules and pure yang-mills   

                                            abstract      label   prompt  \
0  The production of prompt diphotons plays a cru...  synthetic  prompt1   
1  This paper explores the dynamics of a single t...  synthetic  prompt1   
2  This paper investigates the role of curvature ...  synthetic  prompt1   
3  This paper explores the intriguing interplay b...  synthetic  prompt1   
4  In this study, we explore the relationship bet...  synthetic  prompt1   

  categories  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  


In [29]:
directory = 'GPA2Bench' #change to whichever directory
data = []
for file in os.listdir(directory):
    if file.endswith('.csv'):
        file_df = pd.read_csv(os.path.join(directory, file))

        file_name = os.path.splitext(file)[0]
        domain = file_name.split('_')[-1]

        file_df['domain'] = domain
        data.append(file_df)

df = pd.concat(data, ignore_index=True)
df.to_csv(f'{directory}/GPA2Bench_ALL.csv', index=False)


C:\Users\Zeina\AppData\Local\Temp\ipykernel_14996\382968859.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(os.path.join(directory, file))
C:\Users\Zeina\AppData\Local\Temp\ipykernel_14996\382968859.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(os.path.join(directory, file))
C:\Users\Zeina\AppData\Local\Temp\ipykernel_14996\382968859.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(os.path.join(directory, file))


## M4 Dataset


###### Fixing davinci (and also bloomz because its in there for some reason) for peerread (something with newlines, don't really know why this version works. full transparency, this is AI

In [ ]:
import json
import pandas as pd

# Read JSONL line by line
data = []
with open("M4/peerread/peerread_davinci.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        # Optionally clean machine_text or escape newlines here
        # For example:
        if "machine_text" in obj:
            obj["machine_text"] = [s.replace("\n", " ") for s in obj["machine_text"]]
        data.append(obj)

# Save cleaned JSONL
with open("output.jsonl", "w", encoding="utf-8") as f:
    for obj in data:
        json.dump(obj, f, ensure_ascii=False)
        f.write("\n")

# Load with pandas
df = pd.read_json("M4/peerread/peerread_davinci_fixed.jsonl", lines=True)
print(df.head())

In [147]:
import re

with open('M4/peerread/peerread_bloomz.jsonl', 'r', encoding='utf-8') as infile, \
     open('M4/peerread/peerread_bloomz_fixed.jsonl', 'w', encoding='utf-8') as outfile:

    for line in infile:
        # Remove the davinci_reviews field and its value using regex
        # This handles the field appearing anywhere in the line
        cleaned_line = re.sub(
            r',"davinci_reviews":\[.*?\](?=,"|$)',
            '',
            line
        )

        cleaned_line = re.sub(
            r',"chatgpt_reviews":\[.*?\](?=,"|$)',
            '',
            line
        )

        # Also handle if it's the first field (no leading comma)
        cleaned_line = re.sub(
            r'"davinci_reviews":\[.*?\],',
            '',
            cleaned_line
        )

        outfile.write(cleaned_line)

print("Done! Check output.jsonl")

Done! Check output.jsonl


In [148]:

file = pd.read_json("M4/peerread/peerread_bloomz_fixed.jsonl", lines=True)
file.head()

,source,id,title,abstract,human_reviews,prompts,bloom_reviews,score,probas,logits,model
0,PeerRead/acl_2017/,104,Bridge Text and Knowledge by Learning Multi-Pr...,Integrating text and knowledge into a unified ...,[- Strengths:\n* Outperforms ALIGN in supervis...,[Please write a peer review for the paper give...,[The authors propose to learn multi-prototype ...,0.268284,"[0.4606933594, 0.1846923828, 0.1619873047, 0.3...","[19.46875, 17.359375, 18.515625, 22.15625, 18....",bigscience/bloomz
1,PeerRead/acl_2017/,105,Morphological Inflection Generation with Hard ...,We present a neural model for morphological in...,[- Strengths:\nThe idea of hard monotonic atte...,[Please write a peer review for the paper give...,[The authors propose an attention mechanism to...,0.208110,"[0.4658203125, 0.2700195312, 0.1539306641, 0.2...","[19.125, 17.234375, 18.25, 20.1875, 16.625, 19...",bigscience/bloomz
2,PeerRead/acl_2017/,107,Weakly Supervised Cross-Lingual Named Entity R...,The state-of-the-art named entity recognition ...,[This paper presents several weakly supervised...,[Please write a peer review for the paper give...,[The authors propose an effective method to re...,0.254676,"[0.4738769531, 0.3132324219, 0.2371826172, 0.4...","[18.953125, 21.234375, 19.78125, 21.875, 20.57...",bigscience/bloomz
3,PeerRead/acl_2017/,108,A Multigraph-based Model for Overlapping Entit...,"In this paper, we propose a new model for pred...","[- Strengths: the paper is well-written, excep...",[Please write a peer review for the paper give...,[The authors present an interesting approach t...,0.299974,"[0.4094238281, 0.2763671875, 0.1287841797, 0.1...","[19.03125, 18.046875, 18.15625, 19.171875, 26....",bigscience/bloomz
4,PeerRead/acl_2017/,117,Improved Neural Relation Detection for Knowled...,Relation detection is a core component of many...,[- Strengths: The paper addresses a relevant t...,[Please write a peer review for the paper give...,[The authors present an improved neural relati...,0.266504,"[0.4230957031, 0.3024902344, 0.1875, 0.3144531...","[19.28125, 17.953125, 18.6875, 22.21875, 18.40...",bigscience/bloomz


#### looping through files, isolating machine and human text, and creating larger df


In [121]:
all_files = []
directory = 'M4/peerread'
for file in os.listdir(directory):
    if 'bloomz' not in file: #bloomz formatted differently, a lot different col names
        df = pd.read_json(os.path.join(directory, file), lines=True)

        if file == 'peerread_davinci.jsonl':
            continue

        # Create human_text entries
        human_df = df.loc[:, df.columns != 'machine_text'].copy()
        if ('peerread' in file):
            human_df = human_df.explode('human_text')
        human_df.rename(columns={'human_text': 'text'}, inplace=True)
        human_df['model'] = 'not applicable'
        human_df['label'] = 'authentic'

        # Create machine_text entries
        machine_df = df.loc[:, df.columns != 'human_text'].copy()
        if ('peerread' in file):
            machine_df = machine_df.explode('machine_text')
        machine_df.rename(columns={'machine_text': 'text'}, inplace=True)
        machine_df['label'] = 'synthetic'

        # Combine both
        combined = pd.concat([human_df, machine_df], ignore_index=True)

        all_files.append(combined)

# Optional: sort or reset index
final_df = pd.concat(all_files, ignore_index=True)

# fixing some formatting + merge issues
# comment out for wikihow
# final_df.drop(columns=['Unnamed: 0' ], inplace=True)

final_df.to_csv(f'M4/peerread/M4_peerread.csv', encoding='utf-8', errors=False, index=False)

In [122]:
final_df.head()

,prompt,text,model,source,source_ID,label
0,"[Please write a peer review for the paper [], ...",- Strengths:\n* Outperforms ALIGN in supervise...,not applicable,PeerRead/acl_2017/,104,authentic
1,"[Please write a peer review for the paper [], ...",This paper addresses the problem of disambigua...,not applicable,PeerRead/acl_2017/,104,authentic
2,"[Please write a peer review for the paper [], ...","- Strengths:\nGood ideas, simple neural learni...",not applicable,PeerRead/acl_2017/,104,authentic
3,"[Please write a peer review for the paper [], ...",- Strengths:\nThe idea of hard monotonic atten...,not applicable,PeerRead/acl_2017/,105,authentic
4,"[Please write a peer review for the paper [], ...",- Strengths: A new encoder-decoder model is pr...,not applicable,PeerRead/acl_2017/,105,authentic


Fixing BLOOMZ for arXiv

In [110]:
bloomz_df = pd.read_json('M4/arxiv/arxiv_bloomz.jsonl', lines=True)
all_files = pd.read_csv('M4/arxiv/M4_arxiv.csv')

all_files['source_ID'] = all_files['source_ID'].fillna(df['source_id'])
all_files.drop(columns=['source_id'], inplace=True)

# removing unnecessary columns + renaming some columns
# for contect, prompt is the template, machine_text is the prompt with the filled in title and abstract
# kept machine_text
bloomz_df.drop(columns=['probas', 'logits', 'score', 'title', 'prompt'], inplace=True)
bloomz_df.rename(columns={'source_id':'source_ID', 'machine_text': 'prompt'}, inplace=True)

# isolating human data, stored in 'abstract'
human_df = bloomz_df.loc[:, bloomz_df.columns != 'machine_abstract'].copy()
human_df.rename(columns={'abstract': 'text'}, inplace=True)
human_df['model'] = 'not applicable'
human_df['label'] = 'authentic'

# isolating machine data, stored in 'machine_abstract'
machine_df = bloomz_df.loc[:, bloomz_df.columns != 'abstract'].copy()
machine_df.rename(columns={'machine_abstract': 'text'}, inplace=True)
machine_df['label'] = 'synthetic'

# concatenating
all_files = pd.concat([all_files,human_df, machine_df], ignore_index=True)

In [120]:
#removing duplicates, consequence of the way I concatenate. probably a better way to do this
all_files_groups = all_files.groupby('model').size()
print(all_files_groups)

all_files.drop_duplicates(subset=['text'], inplace=True)

all_files_groups = all_files.groupby('model').size()
print(all_files_groups)



(30000, 6)
model
bigscience/bloomz          3000
command-xlarge-nightly     3000
dolly-v2-12b               3000
gpt-3.5-turbo              3000
not applicable            15000
text-davinci-003           3000
dtype: int64


In [122]:
all_files.to_csv(f'M4/arxiv/M4_arxiv.csv', encoding='utf-8', errors=False, index=False)

Fixing BLOOMZ for reddit

In [9]:
bloomz_df = pd.read_json('M4/reddit/reddit_bloomz.jsonl', lines=True)
all_files = pd.read_csv('M4/reddit/M4_reddit.csv')

all_files.drop(columns={'question_id', 'answer_id'}, inplace=True)

bloomz_df.drop(columns=['a_id','score_x', 'probas', 'logits', 'score_y', 'title', 'prompt'],inplace=True)
bloomz_df.rename(columns={'q_id':'source_ID', 'machine_text': 'prompt'}, inplace=True)
bloomz_df['source'] = 'eli5'

# isolating human data, stored in 'abstract'
human_df = bloomz_df.loc[:, bloomz_df.columns != 'machine_answer'].copy()
human_df['model'] = 'not applicable'
human_df['label'] = 'authentic'

# isolating machine data, stored in 'machine_abstract'
machine_df = bloomz_df.loc[:, bloomz_df.columns != 'text'].copy()
machine_df.rename(columns={'machine_answer': 'text'}, inplace=True)
machine_df['label'] = 'synthetic'

all_files = pd.concat([all_files,human_df, machine_df], ignore_index=True)
all_files.head()


,prompt,text,model,source,source_ID,label
0,"I will ask you a question. For this question, ...",Henry died in a joust against the captain of h...,not applicable,eli5,79zzik,authentic
1,"I will ask you a question. For this question, ...","In 1801, James Monroe and Robert R. Livingston...",not applicable,eli5,7p61s8,authentic
2,"I will ask you a question. For this question, ...",Good question! [I answered this a few years ba...,not applicable,eli5,a1z5e9,authentic
3,"I will ask you a question. For this question, ...",Watergate is an incredibly interesting period ...,not applicable,eli5,6wi6jh,authentic
4,"I will ask you a question. For this question, ...","No, in medieval Europe, there were no restric...",not applicable,eli5,9wb4m4,authentic


In [10]:
all_files_groups = all_files.groupby('model').size()
print(all_files_groups)

all_files.drop_duplicates(subset=['text'], inplace=True)

all_files_groups = all_files.groupby('model').size()
print(all_files_groups)

model
bigscience/bloomz     3000
cohere                3000
dolly-v2-12b          3000
gpt-3.5-turbo         3000
not applicable       15000
text-davinci-003      3000
dtype: int64
model
bigscience/bloomz    2999
cohere               1221
dolly-v2-12b         3000
gpt-3.5-turbo        3000
not applicable       3000
text-davinci-003     3000
dtype: int64


In [11]:
all_files.to_csv(f'M4/reddit/M4_reddit.csv', encoding='utf-8', errors=False, index=False)

Fixing Bloomz for wikihow

In [32]:
bloomz_df = pd.read_json('M4/wikihow/wikihow_bloomz.jsonl', lines=True)
all_files = pd.read_csv('M4/wikihow/M4_wikihow.csv')

all_files.drop(columns={'source_ID', 'title', 'headline'}, inplace=True)

bloomz_df.drop(columns=['headline','title', 'probas', 'logits', 'score', 'title', 'prompt'],inplace=True)
bloomz_df.rename(columns={'machine_text': 'prompt'}, inplace=True)

# isolating human data, stored in 'abstract'
human_df = bloomz_df.loc[:, bloomz_df.columns != 'machine_abstract'].copy()
human_df['model'] = 'not applicable'
human_df['label'] = 'authentic'

# isolating machine data, stored in 'machine_abstract'
machine_df = bloomz_df.loc[:, bloomz_df.columns != 'text'].copy()
machine_df.rename(columns={'machine_abstract': 'text'}, inplace=True)
machine_df['label'] = 'synthetic'

all_files = pd.concat([all_files,human_df, machine_df], ignore_index=True)
all_files.head()

,prompt,text,model,source,label
0,"Please, generate wikihow article with length a...",It will work with the Forza Motorsport disc i...,not applicable,wikihow,authentic
1,"Please, generate wikihow article with length a...","They are about $20 a card. Or, if you want to...",not applicable,wikihow,authentic
2,"Please, generate wikihow article with length a...",";\n,,\n\n\nLots of people don't bother to read...",not applicable,wikihow,authentic
3,"Please, generate wikihow article with length a...",Perfumes are a blend of different levels of s...,not applicable,wikihow,authentic
4,"Please, generate wikihow article with length a...","Is it slow? Fast? You have to consider this, ...",not applicable,wikihow,authentic


In [33]:
all_files_groups = all_files.groupby('model').size()
print(all_files_groups)

all_files.drop_duplicates(subset=['text'], inplace=True)

all_files_groups = all_files.groupby('model').size()
print(all_files_groups)

model
bigscience/bloomz          3000
command-xlarge-nightly     3000
dolly-v2-12b               3000
gpt-3.5-turbo              3000
not applicable            15000
text-davinci-003           3000
dtype: int64
model
bigscience/bloomz         3000
command-xlarge-nightly    3000
dolly-v2-12b              3000
gpt-3.5-turbo             3000
not applicable            3003
text-davinci-003          3000
dtype: int64


In [34]:
all_files.to_csv(f'M4/wikihow/M4_wikihow.csv', encoding='utf-8', errors=False, index=False)

Fixing Bloomz for Wikipedia

In [39]:
bloomz_df = pd.read_json('M4/wikipedia/wikipedia_bloomz.jsonl', lines=True)
bloomz_df.head()

,id,url,title,text,machine_abstract,machine_text,prompt,score,probas,logits,model,source
0,William Whitehouse,https://en.wikipedia.org/wiki/William%20Whiteh...,William Whitehouse,William Edward Whitehouse (20 May 1859 – 12 Ja...,The William Whitehouse was the first steam-pow...,Write an abstract for a Wikipedia article with...,Write an abstract for a Wikipedia article with...,0.258311,"[0.0997314453125, 0.19091796875, 0.42236328125...","[15.625, 17.375, 17.875, 18.5625, 17.046875, 1...",bigscience/bloomz,wikipedia
1,Cheryl S. McWatters,https://en.wikipedia.org/wiki/Cheryl%20S.%20Mc...,Cheryl S. McWatters,Cheryl S. McWatters is professor and Father Ed...,"McWatters, Cheryl S. (born March 31, 1953) is ...",Generate an abstract for a Wikipedia article w...,Generate an abstract for a Wikipedia article w...,0.353501,"[0.12054443359375, 0.45849609375, 0.984375, 0....","[15.296875, 19.03125, 26.46875, 18.5625, 18.78...",bigscience/bloomz,wikipedia
2,Lithuanian Lands Militia,https://en.wikipedia.org/wiki/Lithuanian%20Lan...,Lithuanian Lands Militia,The Lithuanian Lands Militia () was a military...,The Lithuanian lands militia was the first nat...,Write an abstract for a Wikipedia article with...,Write an abstract for a Wikipedia article with...,0.341150,"[0.450927734375, 0.90869140625, 0.7685546875, ...","[17.75, 20.75, 21.578125, 17.28125, 20.953125,...",bigscience/bloomz,wikipedia
3,Mizoram–Manipur–Kachin rain forests,https://en.wikipedia.org/wiki/Mizoram%E2%80%93...,Mizoram–Manipur–Kachin rain forests,The Mizoram-Manipur-Kachin rain forests is a s...,The Mizoram-Manipur-Kachin Rain Forests (MMRF)...,Write an abstract for a Wikipedia article with...,Write an abstract for a Wikipedia article with...,0.350881,"[0.5048828125, 0.33154296875, 0.97412109375, 0...","[17.640625, 17.734375, 22.9375, 22.3125, 24.42...",bigscience/bloomz,wikipedia
4,Salesbury,https://en.wikipedia.org/wiki/Salesbury,Salesbury,Salesbury is a village and civil parish in Rib...,The following is the complete list of settleme...,Write an abstract for a Wikipedia article with...,Write an abstract for a Wikipedia article with...,0.295999,"[0.137939453125, 0.058319091796875, 0.29516601...","[16.015625, 15.9140625, 16.671875, 20.1875, 15...",bigscience/bloomz,wikipedia


In [42]:
bloomz_df = pd.read_json('M4/wikipedia/wikipedia_bloomz.jsonl', lines=True)
all_files = pd.read_csv('M4/wikipedia/M4_wikipedia.csv')


bloomz_df.drop(columns=['id','url', 'probas', 'logits', 'score', 'title', 'prompt'],inplace=True)
bloomz_df.rename(columns={'machine_text': 'prompt'}, inplace=True)

# isolating human data, stored in 'abstract'
human_df = bloomz_df.loc[:, bloomz_df.columns != 'machine_abstract'].copy()
human_df['model'] = 'not applicable'
human_df['label'] = 'authentic'

# isolating machine data, stored in 'machine_abstract'
machine_df = bloomz_df.loc[:, bloomz_df.columns != 'text'].copy()
machine_df.rename(columns={'machine_abstract': 'text'}, inplace=True)
machine_df['label'] = 'synthetic'

all_files = pd.concat([all_files,human_df, machine_df], ignore_index=True)
all_files.head()

,prompt,text,model,source,source_id,label
0,"Write a Wikipedia article with the title ""Will...",William Edward Whitehouse (20 May 1859 – 12 Ja...,not applicable,wikipedia-20220301.en,11663875.0,authentic
1,"Write a Wikipedia article with the title ""Cher...",Cheryl S. McWatters is professor and Father Ed...,not applicable,wikipedia-20220301.en,46716205.0,authentic
2,"Write a Wikipedia article with the title ""Mizo...",The Mizoram-Manipur-Kachin rain forests is a s...,not applicable,wikipedia-20220301.en,21243283.0,authentic
3,"Write a Wikipedia article with the title ""Sale...",Salesbury is a village and civil parish in Rib...,not applicable,wikipedia-20220301.en,14717133.0,authentic
4,"Write a Wikipedia article with the title ""Maur...",Sir Maurice Eustace (c. 1590 – 22 June 1665) w...,not applicable,wikipedia-20220301.en,30728425.0,authentic


In [43]:
all_files_groups = all_files.groupby('model').size()
print(all_files_groups)

all_files.drop_duplicates(subset=['text'], inplace=True)

all_files_groups = all_files.groupby('model').size()
print(all_files_groups)

model
bigscience/bloomz         3000
cohere-xlarge-nightly     2336
dolly-v2-12b              2702
gpt-3.5-turbo             2995
not applicable           14033
text-davinci-003          3000
dtype: int64
model
bigscience/bloomz        2999
cohere-xlarge-nightly    2336
dolly-v2-12b             2702
gpt-3.5-turbo            2995
not applicable           4291
text-davinci-003         3000
dtype: int64


In [44]:
all_files.to_csv(f'M4/wikipedia/M4_wikipedia.csv', encoding='utf-8', errors=False, index=False)

Fixing Bloomz for peerread

In [149]:
bloomz_df = pd.read_json('M4/peerread/peerread_bloomz_fixed.jsonl', lines=True)
bloomz_df.head()

,source,id,title,abstract,human_reviews,prompts,bloom_reviews,score,probas,logits,model
0,PeerRead/acl_2017/,104,Bridge Text and Knowledge by Learning Multi-Pr...,Integrating text and knowledge into a unified ...,[- Strengths:\n* Outperforms ALIGN in supervis...,[Please write a peer review for the paper give...,[The authors propose to learn multi-prototype ...,0.268284,"[0.4606933594, 0.1846923828, 0.1619873047, 0.3...","[19.46875, 17.359375, 18.515625, 22.15625, 18....",bigscience/bloomz
1,PeerRead/acl_2017/,105,Morphological Inflection Generation with Hard ...,We present a neural model for morphological in...,[- Strengths:\nThe idea of hard monotonic atte...,[Please write a peer review for the paper give...,[The authors propose an attention mechanism to...,0.208110,"[0.4658203125, 0.2700195312, 0.1539306641, 0.2...","[19.125, 17.234375, 18.25, 20.1875, 16.625, 19...",bigscience/bloomz
2,PeerRead/acl_2017/,107,Weakly Supervised Cross-Lingual Named Entity R...,The state-of-the-art named entity recognition ...,[This paper presents several weakly supervised...,[Please write a peer review for the paper give...,[The authors propose an effective method to re...,0.254676,"[0.4738769531, 0.3132324219, 0.2371826172, 0.4...","[18.953125, 21.234375, 19.78125, 21.875, 20.57...",bigscience/bloomz
3,PeerRead/acl_2017/,108,A Multigraph-based Model for Overlapping Entit...,"In this paper, we propose a new model for pred...","[- Strengths: the paper is well-written, excep...",[Please write a peer review for the paper give...,[The authors present an interesting approach t...,0.299974,"[0.4094238281, 0.2763671875, 0.1287841797, 0.1...","[19.03125, 18.046875, 18.15625, 19.171875, 26....",bigscience/bloomz
4,PeerRead/acl_2017/,117,Improved Neural Relation Detection for Knowled...,Relation detection is a core component of many...,[- Strengths: The paper addresses a relevant t...,[Please write a peer review for the paper give...,[The authors present an improved neural relati...,0.266504,"[0.4230957031, 0.3024902344, 0.1875, 0.3144531...","[19.28125, 17.953125, 18.6875, 22.21875, 18.40...",bigscience/bloomz


In [153]:
bloomz_df = pd.read_json('M4/peerread/peerread_bloomz_fixed.jsonl', lines=True)
all_files = pd.read_csv('M4/peerread/M4_peerread.csv')


bloomz_df.drop(columns=['probas', 'logits', 'score', 'title', 'abstract'],inplace=True)
bloomz_df.rename(columns={'id': 'source_ID', 'prompts': 'prompt'}, inplace=True)

# isolating human data, stored in 'abstract'
human_df = bloomz_df.loc[:, bloomz_df.columns != 'bloom_reviews'].copy()
human_df.rename(columns={'human_reviews': 'text'}, inplace=True)
human_df = human_df.explode('text')
human_df['model'] = 'not applicable'
human_df['label'] = 'authentic'

# isolating machine data, stored in 'machine_abstract'
machine_df = bloomz_df.loc[:, bloomz_df.columns != 'human_reviews'].copy()
machine_df.rename(columns={'bloom_reviews': 'text'}, inplace=True)
machine_df = machine_df.explode('text')
machine_df['label'] = 'synthetic'

all_files = pd.concat([all_files,human_df, machine_df], ignore_index=True)
all_files.head()

,prompt,text,model,source,source_ID,label
0,['Please write a peer review for the paper []'...,- Strengths:\n* Outperforms ALIGN in supervise...,not applicable,PeerRead/acl_2017/,104,authentic
1,['Please write a peer review for the paper []'...,This paper addresses the problem of disambigua...,not applicable,PeerRead/acl_2017/,104,authentic
2,['Please write a peer review for the paper []'...,"- Strengths:\nGood ideas, simple neural learni...",not applicable,PeerRead/acl_2017/,104,authentic
3,['Please write a peer review for the paper []'...,- Strengths:\nThe idea of hard monotonic atten...,not applicable,PeerRead/acl_2017/,105,authentic
4,['Please write a peer review for the paper []'...,- Strengths: A new encoder-decoder model is pr...,not applicable,PeerRead/acl_2017/,105,authentic


In [155]:
all_files_groups = all_files.groupby('model').size()
print(all_files_groups)

all_files.drop_duplicates(subset=['text'], inplace=True)

all_files_groups = all_files.groupby('model').size()
print(all_files_groups)

model
bigscience/bloomz     2340
chatgpt               2344
cohere                2344
davinci               2344
dolly-v2              2344
not applicable       34322
dtype: int64
model
bigscience/bloomz    2334
chatgpt              2344
cohere               2342
davinci              2344
dolly-v2             2344
not applicable       2859
dtype: int64


In [156]:
all_files.to_csv(f'M4/peerread/M4_peerread.csv', encoding='utf-8', errors=False, index=False)

In [163]:
files = ['M4/peerread/M4_peerread.csv', 'M4/wikipedia/M4_wikipedia.csv','M4/wikihow/M4_wikihow.csv','M4/reddit/M4_reddit.csv','M4/arxiv/M4_arxiv.csv']

data =[]

for file in files:
        file_df = pd.read_csv(file)
        data.append(file_df)

final_df = pd.concat(data, ignore_index=True)
final_df['source_id'] = final_df['source_id'].fillna(final_df['source_ID'])
final_df.drop(columns={'source_ID'}, inplace=True)# final_df.to_csv('M4/M4_ALL.csv')
final_df.to_csv(f'M4/M4_ALL.csv', encoding='utf-8', errors=False, index=False)

In [165]:
final_df.head()
print(final_df.shape)

(85111, 6)


## RAID Dataset

In [16]:
raid = load_dataset("liamdugan/raid", on_bad_lines='skip')

Generating train split: 0 examples [00:00, ? examples/s]

Generating extra split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [17]:
train_data = raid['train']
filtered_noattack = train_data.filter(lambda entry : entry.get('attack') == 'none')
filtered_noattack.to_csv("RAID/RAID_noattack.csv")
# filtered_noattack.to_csv("RAID/RAID_noattack")
# filtered_gpt4 = filtered_noattack.filter(lambda entry : any(term in entry.get('model') for term in ['gpt4', 'human']))
# filtered_chatgpt = filtered_noattack.filter(lambda entry : any(term in entry.get('model') for term in ['chatgpt', 'human']))

Filter:   0%|          | 0/5615820 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/468 [00:00<?, ?ba/s]

802130727

In [48]:
df = filtered_gpt4.to_pandas()
df.to_csv("RAID/RAID_gpt4_noattack.csv", index=False)

In [49]:
df = filtered_chatgpt.to_pandas()
df.to_csv("RAID/RAID_chatgpt_noattack.csv", index=False)

In [ ]:
df['label'] = np.where(df['model'] == 'human', 'authentic', 'synthetic')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.to_csv("RAID/RAID_chatgpt_noattack.csv", index=False)

## MAGE Dataset

In [3]:
mage = load_dataset("yaful/MAGE")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


train.csv:   0%|          | 0.00/404M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


valid.csv:   0%|          | 0.00/72.3M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

test_ood_set_gpt.csv: 0.00B [00:00, ?B/s]

test_ood_set_gpt_para.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/319071 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/56792 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/60743 [00:00<?, ? examples/s]

In [13]:
mage_train_data = mage['train'].to_pandas()
mage_train_data['label'] = np.where(mage_train_data['label'] == 1, 'authentic', 'synthetic')
mage_train_data.to_csv("MAGE/MAGE_(train_only).csv", index=False)


In [170]:
file = pd.read_csv('MAGE/MAGE_(train_only)_sample.csv')
print(file['domain'].unique())
print(len(file['model'].unique()))

['cmv' 'eli5' 'tldr' 'xsum' 'wp' 'roct' 'hswag' 'yelp' 'squad' 'sci_gen']
34


In [173]:
file = pd.read_csv('M4/M4_ALL.csv')
print(len(file['source'].unique()))
print(len(file['model'].unique()))

9
11


In [174]:
file = pd.read_csv('RAID/RAID_noattack_sample.csv')
print(file['domain'].unique())
print(len(file['model'].unique()))

['abstracts' 'books' 'news' 'poetry' 'recipes' 'reddit' 'reviews' 'wiki']
12
